### Making TIMBR predictions using RIPTIDE

Only use one model and place very small constraints on lower bounds to achieve one flux distribution to make predictions for each metabolite

In [1]:
import cobra.io
import numpy
from riptide import *

In [2]:
# Change to correct directory to access data folder
os.chdir("..")
os.chdir("..")

In [3]:
# Translated model from iCardio
heart_model = cobra.io.load_matlab_model("data/iRno_heart.mat", 
                                        variable_name = "rno_heart_model")

# Load in original general network reconstruction
rno_model = cobra.io.load_matlab_model("data/iRno_heart.mat", 
                                      variable_name = "rno_cobra_load")

In [4]:
model = heart_model
model.objective = "RCR11017"

In [5]:
# Change the upper bounds of reactions from 1000 to 10e6 as was done in the TIMBR paper
for reaction in model.reactions:
    if reaction.upper_bound == 1000:
        reaction.upper_bound = 10e6

In [6]:
# Identify reactions that have an upper and lower bound of 0 in the model
to_remove = []
for x in model.reactions:
    if x.lower_bound == 0 and x.upper_bound == 0:
        to_remove.append(x)

model.remove_reactions(to_remove)

In [7]:
# Add back reactions from the iRno model to the heart model that are necessary
# for transport reactions for media conditions
# and transport reactions for biomarker predictions
add_rxns = ['RCR30349', 'RCR30125', 'RCR30118', 'RCR30055', 'RCR30068', \
    'RCR30053', 'RCR30436', 'RCR30233', 'RCR30149', 'RCR30199', 'RCR30078', \
    'RCR30460', 'RCR30002', 'RCR30027', 'RCR30041', 'RCR30054', 'RCR30071', \
    'RCR30080', 'RCR30086', 'RCR30107', 'RCR30120', \
    'RCR30131', 'RCR30148', 'RCR30150', 'RCR30160', 'RCR30191', \
    'RCR30216', 'RCR30218', 'RCR30228', 'RCR30234', 'RCR30238', 'RCR30374', \
    'RCR30410', 'RCR30438', 'RCR30481', 'RCR30555','RCR30239', \
    'RCR90192', 'RCR90193']

for x in add_rxns:
    model.add_reaction(rno_model.reactions.get_by_id(x))

In [8]:
# Exchange reactions that were defined based on physiological bounds in the iRno model
exchange_rxns = ['RCR30013', 'RCR30014', 'RCR30015', 'RCR30019', 'RCR30024', \
                 'RCR30030', 'RCR30031', 'RCR30035', 'RCR30042', 'RCR30043', \
                 'RCR30045', 'RCR30095', 'RCR30106', 'RCR30110', 'RCR30116', \
                 'RCR30117', 'RCR30119', 'RCR30122', 'RCR30150', 'RCR30183', \
                 'RCR30184', 'RCR30185', 'RCR30188', 'RCR30199', 'RCR30315', \
                 'RCR30317', 'RCR30318', 'RCR30321', 'RCR30327', 'RCR30348', \
                 'RCR30349', 'RCR30399', 'RCR30461', 'RCR30527', 'RCR30528', \
                 'RCR30529', 'RCR30530', 'RCR30531', 'RCR30539', 'RCR90123', \
                 'RCR90202', 'RCR90203', 'RCR90206', 'RCR90210']

# Reactions that aren't present were removed during model curation
for x in exchange_rxns:
    try:
        # first change bounds on rno_model
        model.reactions.get_by_id(x).lower_bound = 0.
    except:
        print(x, 'not found')

RCR30043 not found
RCR30119 not found
RCR30399 not found
RCR30461 not found


In [9]:
# Define media conditions from in vitro experiments
# Consumed metabolites
media_composition = {'RCR30032','RCR30349','RCR30125','RCR30106','RCR30068',\
                    'RCR30053','RCR30348','RCR30460',\
                    'RCR30211','RCR30077','RCR30311','RCR30536'}

# media_composition = {'RCR30032', 'RCR30211', 'RCR30013', 'RCR30349', 'RCR30125', \
#     'RCR30015', 'RCR30145', 'RCR30106', 'RCR30118', 'RCR30095', \
#     'RCR30055', 'RCR30068', 'RCR30030', 'RCR30315', 'RCR30005', \
#     'RCR30321', 'RCR30116', 'RCR30053', 'RCR30077', 'RCR30527', \
#     'RCR30070', 'RCR30436', 'RCR30528', 'RCR30529', 'RCR30530', \
#     'RCR30531', 'RCR30233', 'RCR30348', 'RCR30184', 'RCR30117', \
#     'RCR30110', 'RCR30149', 'RCR30536', 'RCR30199', 'RCR30078', \
#     'RCR30042', 'RCR30317', 'RCR30072', 'RCR30318', 'RCR30031', \
#     'RCR30237', 'RCR30185', 'RCR30460'}

for x in media_composition:
    model.reactions.get_by_id(x).lower_bound = -10.
    model.reactions.get_by_id(x).upper_bound = 0.

# Change the lower bound 
model.reactions.get_by_id('RCR90192').lower_bound = 1
model.reactions.get_by_id('RCR90193').lower_bound = 1

In [10]:
# noChange = {'RCR30160','RCR30015','RCR30145',\
#             'RCR30228','RCR30041',\
#             'RCR30311','RCR30142','RCR30118', \
#             'RCR30095','RCR30055','RCR30030','RCR30148','RCR30315',\
#             'RCR30321','RCR30027','RCR30116','RCR30527',\
#             'RCR30528','RCR30239','RCR30529','RCR30530','RCR30438',\
#             'RCR30071','RCR30120','RCR30531',\
#             'RCR30436','RCR30233','RCR30019','RCR30010','RCR30184',\
#             'RCR30117',\
#             'RCR30234','RCR30374','RCR30149','RCR30199',\
#             'RCR30078','RCR30042',\
#             'RCR30322','RCR30150','RCR30317','RCR30072','RCR30318',\
#             'RCR30031','RCR30237','RCR30185'}
# for x in noChange:
#     model.reactions.get_by_id(x).bounds = (0.,0.)

# print(model.optimize().objective_value)

In [11]:
# identify set of metabolites that can be produced given the media constraints
# Create a set of the exchange reactions in the model
exchanges = set()
for reaction in model.boundary:
    if reaction.bounds != (0., 0.):
        exchanges.add(reaction.name)
        
# remove metabolites in the media since placing a lower bound on production
exchanges = exchanges - media_composition

# Run an FBA for each metabolite in exchange set
# Include metabolite if FBA > 1e-4
predictions = set()
for reaction in exchanges: 
    model.objective = reaction
    if model.optimize().objective_value > 1:
        predictions.add(reaction)

# Number of metabolites for predictions drops to 84 when excluding media metabolites
print(len(predictions))

87


In [12]:
produced = {'RCR30191','RCR30086','RCR30218','RCR30013',\
            'RCR30033','RCR30014','RCR30107','RCR30016',\
            'RCR30216','RCR30410','RCR30005','RCR30002',\
            'RCR30070','RCR30054','RCR30024',\
            'RCR30188','RCR30481','RCR30533',\
            'RCR30110',\
            'RCR30555','RCR30080','RCR30131','RCR30238'}

print(len(produced))
print(len(list(predictions & produced)))

produced = list(predictions & produced)
for x in produced:
    model.reactions.get_by_id(x).lower_bound = 1
model.objective = "RCR11017"
print(model.optimize().objective_value)

23
13
700.2946017069704


In [13]:
# Load in data for individual conditions: 
# 5FU_24: sample1-sample6
# 5FU_6: sample7-sample12
# Ace_24: sample13-sample19
# Ace_6: sample20-sample25
# DMSO1_24: sample26-sample31
# DMSO1_6: sample32 - sample37
# DMSO2_24: sample38 - sample44
# DMSO2_6: sample45 - sample51
# Dox_24: sample52 - sample58
# Dox_6: sample59 - sample65

fiveFU_6 = {}
fiveFU_24 = {}
ace_6 = {}
ace_24 = {}
dmso1_6 = {}
dmso1_24 = {}
dmso2_6 = {}
dmso2_24 = {}
dox_24 = {}
dox_6 = {}
with open('data/RNA-seq/scaledTPMs.txt', 'r') as transcription:
    for line in transcription:
        line = line.split()
        if line[0].strip('"') == 'NAME':
            continue
        else:
            fiveFU_24[line[0].strip('"')] = numpy.median([numpy.float(x) for x in line[2:8]])
            fiveFU_6[line[0].strip('"')] = numpy.median([numpy.float(x) for x in line[8:14]])
            ace_24[line[0].strip('"')] = numpy.median([numpy.float(x) for x in line[14:21]])
            ace_6[line[0].strip('"')] = numpy.median([numpy.float(x) for x in line[21:27]])
            dmso1_24[line[0].strip('"')] = numpy.median([numpy.float(x) for x in line[27:33]])
            dmso1_6[line[0].strip('"')] = numpy.median([numpy.float(x) for x in line[33:39]])
            dmso2_24[line[0].strip('"')] = numpy.median([numpy.float(x) for x in line[39:46]])
            dmso2_6[line[0].strip('"')] = numpy.median([numpy.float(x) for x in line[46:53]])
            dox_24[line[0].strip('"')] = numpy.median([numpy.float(x) for x in line[53:60]])
            dox_6[line[0].strip('"')] = numpy.median([numpy.float(x) for x in line[60:67]])

# print an example to make sure medians are correct
print(fiveFU_24.get("24152"))
print(fiveFU_24.get("24170"))

0.0
1660.983030116855


In [14]:
# Ensure that the model can still produce a feasible flux distribution
model.objective = "RCR11017"
model.reactions.get_by_id("RCR11017").bounds = (0.,100.)
print(model.optimize().objective_value)

100.0


In [15]:
data = riptide.contextualize(model, fraction = 0.9, samples = 100, transcriptome = dox_6)
riptide.save_output(data, path = "data/RIPTIDE/dox_6h")


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 190 from 4249 (95.53% change)
Metabolites pruned to 191 from 2920 (93.46% change)
Context-specific metabolism correlates with transcriptome (r=0.366, p<0.001 *)

RIPTiDe completed in, 2 minutes and 19 seconds 

